In [1]:
!pip install autogluon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 48 kB 4.9 MB/s 
     |████████████████████████████████| 210 kB 27.8 MB/s 
     |████████████████████████████████| 65 kB 4.0 MB/s 
     |████████████████████████████████| 149 kB 67.2 MB/s 
     |████████████████████████████████| 59 kB 7.8 MB/s 
     |████████████████████████████████| 274 kB 60.7 MB/s 
     |████████████████████████████████| 61 kB 493 kB/s 
     |████████████████████████████████| 1.0 MB 52.1 MB/s 
     |████████████████████████████████| 802 kB 49.8 MB/s 
     |████████████████████████████████| 132 kB 52.9 MB/s 
     |████████████████████████████████| 1.6 MB 18.0 MB/s 
     |████████████████████████████████| 54.5 MB 1.2 MB/s 
     |████████████████████████████████| 291 kB 62.6 MB/s 
     |████████████████████████████████| 4.3 MB 53.6 MB/s 
     |████████████████████████████████| 74 kB 3.6 MB/s 
     |████████████████████████████████| 4

In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.metrics import classification_report
import pandas as pd


# **Performance of App Recognition on NordVPN Dataset (CV1)**

In [2]:
df_train = pd.read_csv(f'NordVPN_CV1_train.csv', )
df_test = pd.read_csv(f'NordVPN_CV1_test.csv', )

#df_train = pd.read_csv(f'../dataset/NordVPN/NordVPN_CV1_train.csv', )
#df_test = pd.read_csv(f'../dataset/NordVPN/NordVPN_CV1_test.csv', )

title_small = []

### Feature Vector Size : 1,670 (average size of packet length sequnece in train set.)
for i in range(1670):
  title_small.append(str(i))

df_train[title_small] = (df_train['Packet_Length_Sequence'].str.split(' ', expand=True)).drop(list(range(1670, 10000)), axis='columns')
df_test[title_small] = (df_test['Packet_Length_Sequence'].str.split(' ', expand=True)).drop(list(range(1670, 10000)), axis='columns')


df_train = df_train.drop(labels=['Packet_Length_Sequence'], axis='columns')   
df_test = df_test.drop(labels=['Packet_Length_Sequence'], axis='columns')   

save_path = 'ag_models/'

predictor = TabularPredictor(label="Label", problem_type='multiclass', eval_metric='f1_weighted', path=save_path).fit(df_train, time_limit=600)

results = predictor.fit_summary(show_plot=True)

y_test = df_test['Label']    
df_test = df_test.drop(labels=['Label'], axis=1)   
y_pred = predictor.predict(df_test)
y_pred_prob = predictor.predict_proba(df_test)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)
labels_100 = [int(i) for i in range(100)]
print(classification_report(y_test, y_pred, labels=labels_100, digits=4))

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[k1] = value[k2]
INFO:autogluon.tabular.learner.default_learner:Beginning AutoGluon training ... Time limit = 600s
INFO:autogluon.tabular.learner.default_learner:AutoGluon will save models to "ag_models/"
INFO:autogluon.tabular.learner.default_learner:AutoGluon Version:  0.5.2
INFO:autogluon.tabular.learner.default_learner:Python Version:     3.7.15
INFO:autogluon.tabular.learner.default_learner:Operating System:   Linux
INFO:autogluon.tabular.learner.default_learner:Train Data Rows:    1000
INFO:autogluon.tabular.learner.default_learner:Train Data Columns: 1670
INFO:autogluon.tabular.learner.default_learner:Label Column: Label
INFO:autogluon

[1000]	valid_set's multi_logloss: 2.81629	valid_set's f1_weighted: 0.315429


INFO:autogluon.core.trainer.abstract_trainer:	0.3274	 = Validation score   (f1_weighted)
INFO:autogluon.core.trainer.abstract_trainer:	477.26s	 = Training   runtime
INFO:autogluon.core.trainer.abstract_trainer:	2.72s	 = Validation runtime
INFO:autogluon.core.trainer.abstract_trainer:Fitting model: LightGBM ... Training model for up to 44.22s of the 44.18s of remaining time.
INFO:autogluon.tabular.models.lgb.callbacks:	Ran out of time, early stopping on iteration 87. Best iteration is:
	[87]	valid_set's multi_logloss: 3.67173	valid_set's f1_weighted: 0.19569
INFO:autogluon.core.trainer.abstract_trainer:	0.1957	 = Validation score   (f1_weighted)
INFO:autogluon.core.trainer.abstract_trainer:	45.83s	 = Training   runtime
INFO:autogluon.core.trainer.abstract_trainer:	1.51s	 = Validation runtime
INFO:autogluon.core.trainer.abstract_trainer:Fitting model: WeightedEnsemble_L2 ... Training model for up to 360.0s of the -3.76s of remaining time.
INFO:autogluon.core.trainer.abstract_trainer:	0.5

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2   0.587635       4.287366  535.336953                0.001533           0.675320            2       True          4
1      NeuralNetFastAI   0.562048       1.567548   57.400512                1.567548          57.400512            1       True          1
2           LightGBMXT   0.327357       2.718284  477.261121                2.718284         477.261121            1       True          2
3             LightGBM   0.195690       1.505276   45.825422                1.505276          45.825422            1       True          3
Number of models trained: 4
Types of models trained:
{'LGBModel', 'WeightedEnsembleModel', 'NNFastAiTabularModel'}
Bagging used: False 
Multi-layer stack-ensembling used: False 
Feature Metadata (Processed):
(raw dtype, special dtypes):
('cate

INFO:autogluon.tabular.learner.abstract_learner:Evaluation: f1_weighted on test data: 0.6260196887118942
INFO:autogluon.tabular.learner.abstract_learner:Evaluations on test data:
INFO:autogluon.tabular.learner.abstract_learner:{
    "f1_weighted": 0.6260196887118942,
    "accuracy": 0.642,
    "balanced_accuracy": 0.6420000000000001,
    "mcc": 0.639275571731759
}


              precision    recall  f1-score   support

           0     0.7000    0.7000    0.7000        10
           1     0.6667    0.8000    0.7273        10
           2     0.6250    1.0000    0.7692        10
           3     0.4615    0.6000    0.5217        10
           4     0.6154    0.8000    0.6957        10
           5     0.8571    0.6000    0.7059        10
           6     1.0000    0.9000    0.9474        10
           7     0.8333    0.5000    0.6250        10
           8     0.7500    0.6000    0.6667        10
           9     1.0000    0.9000    0.9474        10
          10     0.7778    0.7000    0.7368        10
          11     0.6429    0.9000    0.7500        10
          12     0.6000    0.3000    0.4000        10
          13     1.0000    0.8000    0.8889        10
          14     0.8333    0.5000    0.6250        10
          15     0.7500    0.6000    0.6667        10
          16     0.7143    1.0000    0.8333        10
          17     0.4615    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# **Feature Vector Size Evaluation (Size of Packet Length Sequence) - from 500 to 10000**

In [ ]:
result_txt = open(f"result_NordVPN_cutting_size_CV1.txt", 'w')

for cutting_size in range(500, 10500, 500):
  df_train = pd.read_csv(f'NordVPN_CV1_train.csv', )
  df_test = pd.read_csv(f'NordVPN_CV1_test.csv', )

  title_small = []

  for i in range(cutting_size):
    title_small.append(str(i))

  df_train[title_small] = (df_train['Packet_Length_Sequence'].str.split(' ', expand=True)).drop(list(range(cutting_size, 10000)), axis='columns')
  df_test[title_small] = (df_test['Packet_Length_Sequence'].str.split(' ', expand=True)).drop(list(range(cutting_size, 10000)), axis='columns')


  df_train = df_train.drop(labels=['Packet_Length_Sequence'], axis='columns')   
  df_test = df_test.drop(labels=['Packet_Length_Sequence'], axis='columns')   

  save_path = 'ag_models/'

  predictor = TabularPredictor(label="Label", problem_type='multiclass', eval_metric='f1_weighted', path=save_path).fit(df_train, time_limit=600)

  results = predictor.fit_summary(show_plot=True)

  y_test = df_test['Label']    
  df_test = df_test.drop(labels=['Label'], axis=1)   
  y_pred = predictor.predict(df_test)
  y_pred_prob = predictor.predict_proba(df_test)
  perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)
  labels_100 = [int(i) for i in range(100)]
  print(classification_report(y_test, y_pred, labels=labels_100, digits=4))

  result_txt.write(f"Cutting_size : {cutting_size}\n")
  result_txt.write(classification_report(y_test, y_pred, labels=labels_100, digits=4))

result_txt.close()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[k1] = value[k2]
INFO:autogluon.tabular.learner.default_learner:Beginning AutoGluon training ... Time limit = 600s
INFO:autogluon.tabular.learner.default_learner:AutoGluon will save models to "ag_models/"
INFO:autogluon.tabular.learner.default_learner:AutoGluon Version:  0.5.2
INFO:autogluon.tabular.learner.default_learner:Python Version:     3.7.15
INFO:autogluon.tabular.learner.default_learner:Operating System:   Linux
INFO:autogluon.tabular.learner.default_learner:Train Data Rows:    1000
INFO:autogluon.tabular.learner.default_learner:Train Data Columns: 500
INFO:autogluon.tabular.learner.default_learner:Label Column: Label
INFO:autogluon.

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2   0.443929       2.308801  1218.345001                0.001588           1.132007            2       True          7
1      NeuralNetFastAI   0.436167       0.280591    20.054441                0.280591          20.054441            1       True          1
2           LightGBMXT   0.246714       0.504224   122.520819                0.504224         122.520819            1       True          2
3             LightGBM   0.232127       0.494443   125.570919                0.494443         125.570919            1       True          3
4     RandomForestGini   0.207738       0.448609     5.965704                0.448609           5.965704            1       True          4
5     RandomForestEntr   0.155571       0.425511     8.879951                0.425511           8.

INFO:autogluon.tabular.learner.abstract_learner:Evaluation: f1_weighted on test data: 0.4608481522182845
INFO:autogluon.tabular.learner.abstract_learner:Evaluations on test data:
INFO:autogluon.tabular.learner.abstract_learner:{
    "f1_weighted": 0.4608481522182845,
    "accuracy": 0.484,
    "balanced_accuracy": 0.48399999999999993,
    "mcc": 0.4795096238118434
}
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/loca

              precision    recall  f1-score   support

           0     0.0833    0.1000    0.0909        10
           1     0.3810    0.8000    0.5161        10
           2     0.8000    0.8000    0.8000        10
           3     0.7143    0.5000    0.5882        10
           4     0.4000    0.2000    0.2667        10
           5     0.2000    0.5000    0.2857        10
           6     0.5294    0.9000    0.6667        10
           7     0.2857    0.2000    0.2353        10
           8     0.5333    0.8000    0.6400        10
           9     0.7000    0.7000    0.7000        10
          10     0.8889    0.8000    0.8421        10
          11     1.0000    0.4000    0.5714        10
          12     0.4545    0.5000    0.4762        10
          13     0.6667    0.4000    0.5000        10
          14     0.5000    0.1000    0.1667        10
          15     0.3333    0.1000    0.1538        10
          16     0.4375    0.7000    0.5385        10
          17     0.3000    

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[k1] = value[k2]
INFO:autogluon.tabular.learner.default_learner:Beginning AutoGluon training ... Time limit = 600s
INFO:autogluon.tabular.learner.default_learner:AutoGluon will save models to "ag_models/"
INFO:autogluon.tabular.learner.default_learner:AutoGluon Version:  0.5.2
INFO:autogluon.tabular.learner.default_learner:Python Version:     3.7.15
INFO:autogluon.tabular.learner.default_learner:Operating System:   Linux
INFO:autogluon.tabular.learner.default_learner:Train Data Rows:    1000
INFO:autogluon.tabular.learner.default_learner:Train Data Columns: 1000
INFO:autogluon.tabular.learner.default_learner:Label Column: Label
INFO:autogluon

...

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[k1] = value[k2]
INFO:autogluon.tabular.learner.default_learner:Beginning AutoGluon training ... Time limit = 600s
INFO:autogluon.tabular.learner.default_learner:AutoGluon will save models to "ag_models/"
INFO:autogluon.tabular.learner.default_learner:AutoGluon Version:  0.5.2
INFO:autogluon.tabular.learner.default_learner:Python Version:     3.7.15
INFO:autogluon.tabular.learner.default_learner:Operating System:   Linux
INFO:autogluon.tabular.learner.default_learner:Train Data Rows:    1000
INFO:autogluon.tabular.learner.default_learner:Train Data Columns: 8500
INFO:autogluon.tabular.learner.default_learner:Label Column: Label
INFO:autogluon

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2   0.595952      47.692584  493.421978                0.002007           0.471973            2       True          3
1      NeuralNetFastAI   0.595000      24.240225  268.838409               24.240225         268.838409            1       True          1
2           LightGBMXT   0.301659      23.450352  224.111595               23.450352         224.111595            1       True          2
Number of models trained: 3
Types of models trained:
{'LGBModel', 'WeightedEnsembleModel', 'NNFastAiTabularModel'}
Bagging used: False 
Multi-layer stack-ensembling used: False 
Feature Metadata (Processed):
(raw dtype, special dtypes):
('category', []) : 8500 | ['0', '1', '2', '3', '4', ...]
Plot summary of models saved to file: ag_models/SummaryOfModels.html
*** End of fit() su

INFO:autogluon.tabular.learner.abstract_learner:Evaluation: f1_weighted on test data: 0.6415286344374108
INFO:autogluon.tabular.learner.abstract_learner:Evaluations on test data:
INFO:autogluon.tabular.learner.abstract_learner:{
    "f1_weighted": 0.6415286344374108,
    "accuracy": 0.658,
    "balanced_accuracy": 0.6579999999999999,
    "mcc": 0.6552301638071941
}
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local

              precision    recall  f1-score   support

           0     0.5000    0.8000    0.6154        10
           1     0.9000    0.9000    0.9000        10
           2     0.9091    1.0000    0.9524        10
           3     1.0000    1.0000    1.0000        10
           4     0.7000    0.7000    0.7000        10
           5     0.5556    0.5000    0.5263        10
           6     1.0000    0.9000    0.9474        10
           7     0.7500    0.6000    0.6667        10
           8     0.8750    0.7000    0.7778        10
           9     1.0000    0.9000    0.9474        10
          10     0.8333    0.5000    0.6250        10
          11     0.6429    0.9000    0.7500        10
          12     1.0000    0.1000    0.1818        10
          13     0.7143    0.5000    0.5882        10
          14     0.6000    0.6000    0.6000        10
          15     0.2083    0.5000    0.2941        10
          16     1.0000    1.0000    1.0000        10
          17     0.8571    

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[k1] = value[k2]
INFO:autogluon.tabular.learner.default_learner:Beginning AutoGluon training ... Time limit = 600s
INFO:autogluon.tabular.learner.default_learner:AutoGluon will save models to "ag_models/"
INFO:autogluon.tabular.learner.default_learner:AutoGluon Version:  0.5.2
INFO:autogluon.tabular.learner.default_learner:Python Version:     3.7.15
INFO:autogluon.tabular.learner.default_learner:Operating System:   Linux
INFO:autogluon.tabular.learner.default_learner:Train Data Rows:    1000
INFO:autogluon.tabular.learner.default_learner:Train Data Columns: 9000
INFO:autogluon.tabular.learner.default_learner:Label Column: Label
INFO:autogluon